In [29]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HealthcareETL").getOrCreate()

file_path = "healthcare_dataset.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [30]:
df.show()

+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|
|       LesLie TErRy| 62|  Male|        A+|          Obesity|       2019-08-20| Samantha Davies|            

In [31]:
total_columns = len(df.columns)
print(f"Total number of columns: {total_columns}")

total_rows = df.count()
print(f"Total number of rows: {total_rows}")

duplicate_count = total_rows - df.dropDuplicates().count()
print(f"Total number of duplicate rows: {duplicate_count}")

Total number of columns: 15
Total number of rows: 55500
Total number of duplicate rows: 534


In [ ]:
from pyspark.sql.functions import col, trim, initcap, lower, to_date, when, lit

# Normalize Name
cleaned_df = df.withColumn("Name", initcap(col("Name")))

# Standardize Gender
cleaned_df = cleaned_df.withColumn("Gender", lower(trim(col("Gender"))))

# Convert Dates
cleaned_df = cleaned_df.withColumn("Date of Admission", to_date(col("Date of Admission"), "yyyy-MM-dd"))
cleaned_df = cleaned_df.withColumn("Discharge Date", to_date(col("Discharge Date"), "yyyy-MM-dd"))

# Fill Missing Values
from pyspark.sql.functions import mean

# Fill missing Billing Amount with the median
median_value = cleaned_df.approxQuantile("Billing Amount", [0.5], 0.0)[0]
cleaned_df = cleaned_df.fillna({"Billing Amount": median_value})

# Fill missing categorical values
cleaned_df = cleaned_df.fillna({
    "Medical Condition": "Unknown",
    "Doctor": "Unknown",
    "Hospital": "Unknown",
    "Insurance Provider": "Unknown",
    "Admission Type": "Unknown",
    "Medication": "Unknown",
    "Test Results": "Unknown"
})

In [33]:
# Remove invalid ages
cleaned_df = cleaned_df.filter((col("Age") > 0) & (col("Age") <= 120))

# Remove records with missing Room Number
cleaned_df = cleaned_df.na.drop(subset=["Room Number"])

# Aggregate Billing Amount by Hospital
from pyspark.sql.functions import sum

billing_summary = cleaned_df.groupBy("Hospital").agg(sum("Billing Amount").alias("Total Billing"))
billing_summary.show(truncate=False)

+-------------------------+------------------+
|Hospital                 |Total Billing     |
+-------------------------+------------------+
|Ramirez-Robinson         |96739.00128614204 |
|Foster Lamb, Graham and  |10659.608812944593|
|LLC Massey               |71446.9940966842  |
|Coleman-Aguilar          |34961.90671326528 |
|Group Stein              |25094.95572621151 |
|Smith PLC                |1029424.449116314 |
|Alvarado-Martin          |1077.1696809399066|
|Hall Group               |247201.21558231176|
|and Mayo Chen, Murray    |49400.214028011644|
|Lopez-Wilson             |56481.55593873338 |
|Harris-Farrell           |32166.967383439547|
|Dawson-Williams          |9616.626025733382 |
|Holmes Reed and Johnson, |9492.225487246009 |
|and Lee Rodriguez Morris,|18047.5510046168  |
|Watkins, and Young Perry |41838.22082511615 |
|Nunez-Hamilton           |40446.09173896364 |
|Freeman-Hunter           |29414.68600622586 |
|and Aguilar Sons         |92686.82176385348 |
|Reeves-Edwar

In [34]:
from pyspark.sql.functions import count, col

# Find and show duplicate rows
duplicates_df = df.groupBy(df.columns).agg(count("*").alias("count")).filter(col("count") > 1).drop("count")
duplicates_df.show(truncate=False)

+--------------------+---+------+----------+-----------------+-----------------+---------------------+------------------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|Name                |Age|Gender|Blood Type|Medical Condition|Date of Admission|Doctor               |Hospital                      |Insurance Provider|Billing Amount    |Room Number|Admission Type|Discharge Date|Medication |Test Results|
+--------------------+---+------+----------+-----------------+-----------------+---------------------+------------------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|maTthEw SEllERS     |71 |Female|O-        |Cancer           |2021-02-06       |Erin Page            |Brown-Cortez                  |UnitedHealthcare  |47566.507842139574|400        |Emergency     |2021-02-10    |Ibuprofen  |Abnormal    |
|samUel brYaNt       |53 |Male  |O+        |

In [35]:

cleaned_df = cleaned_df.dropDuplicates()

cleaned_df.show(truncate=False)

+------------------+---+------+----------+-----------------+-----------------+-------------------+------------------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|Name              |Age|Gender|Blood Type|Medical Condition|Date of Admission|Doctor             |Hospital                      |Insurance Provider|Billing Amount    |Room Number|Admission Type|Discharge Date|Medication |Test Results|
+------------------+---+------+----------+-----------------+-----------------+-------------------+------------------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|James Ross        |83 |female|A+        |Diabetes         |2024-01-13       |Michael Baker      |Cox-Hester                    |Blue Cross        |10352.20848674086 |394        |Urgent        |2024-01-22    |Aspirin    |Abnormal    |
|Andrea Allen      |47 |female|AB+       |Obesity          |

In [36]:

total_columns = len(cleaned_df.columns)
print(f"Total number of columns: {total_columns}")

total_rows = cleaned_df.count()
print(f"Total number of rows: {total_rows}")

duplicate_count = total_rows - df.dropDuplicates().count()
print(f"Total number of duplicate rows: {duplicate_count}")

Total number of columns: 15
Total number of rows: 54966
Total number of duplicate rows: 0


In [39]:
output_path = "cleaned_data.csv"
cleaned_df.coalesce(1).write.mode("overwrite").csv(output_path, header=True)

Py4JJavaError: An error occurred while calling o435.csv.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOu